In [ ]:
def Rank(s, r):
    """
    Input: input string s; minimum rank r
    Output: rank array R = {(pi, Ri )}: pi denotes the index of the letter in
    s corresponding to the i-th pair, and Ri is its rank
    """
    